In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Robot
import time
robot = Robot()
controller = widgets.Controller(index=0)  # replace with index of your controller
display(controller)

In [ ]:
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward1', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)
fright_button = widgets.Button(description='fright', layout=button_layout)
fleft_button = widgets.Button(description='fleft', layout=button_layout)
bright_button = widgets.Button(description='bright', layout=button_layout)
bleft_button = widgets.Button(description='bleft', layout=button_layout)

# display buttons
middle_box1 = widgets.HBox([fleft_button,forward_button , fright_button], layout=widgets.Layout(align_self='center'))
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
middle_box2 = widgets.HBox([bleft_button, backward_button, bright_button], layout=widgets.Layout(align_self='center'))

controls_box = widgets.VBox([middle_box1, middle_box, middle_box2])


In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Robot
import traitlets
import time
robot = Robot()
controller = widgets.Controller(index=0)  # replace with index of your controller
display(controller)

In [ ]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg
camera = Camera.instance()
image = widgets.Image(format='jpeg', width=300, height=300)
origin_image = widgets.Image(format='jpeg', width=300, height=300)
diff_image = widgets.Image(format='jpeg', width=300, height=300)


In [ ]:
import uuid
import subprocess
import cv2 as cv, numpy as np
import time
subprocess.call(['mkdir', '-p', 'snapshots'])

snapshot_image = widgets.Image(format='jpeg', width=300, height=300)

def frame_diff(prev_frame, cur_frame):
    return_diff = cv.absdiff(cur_frame, prev_frame)
    
    threshold = len(return_diff[np.where(return_diff > 2)])

#     if threshold > 500: print('threshold > 200 : ', threshold)

    return return_diff

def frame_hue(img):    
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    h, s, v = cv.split(hsv)
    s = cv.inRange(s, 20, 200)
    dark = cv.bitwise_and(hsv,hsv,mask=s)
    dark[dark[:,:,1]<100] = 0
#     dark = cv.cvtColor(dark, cv.COLOR_HSV2BGR)
    
    return dark

    
def getContours(origin,gray):    
    ori= origin  
    approx = 0
    contours, hierarchy = cv.findContours(gray, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)

    for cnt in contours:
        area = cv.contourArea(cnt)
        peri = cv.arcLength(cnt, True)
        approx = cv.approxPolyDP(cnt, 0.02*peri, True)
        
        if  area > 100 and area < 1600:
#             peri = cv.arcLength(cnt, True)
#             approx = cv.approxPolyDP(cnt, 0.02*peri, True)
            if len(approx) < 10  :
                cv.drawContours(ori, cnt, -1, (0, 0, 255), 1)         
                x, y, w, h = cv.boundingRect(approx)
                cv.putText(img,f"axis :{len(approx)}",(y+10,x-10),cv.FONT_HERSHEY_COMPLEX,0.5,(255,0,0),1)
#         elif approx:
#             cv.drawContours(ori, cnt, -1, (0, 0, 0), cv.FILLED)
#         else:
#             cv.drawContours(ori, cnt, -1, (0, 0, 0), cv.FILLED)
# #             cv.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), -1)  
    return (ori,approx)

def convert_to_gray(origin_img):
 
    img = cv.cvtColor(origin_img, cv.COLOR_BGR2GRAY)
    _,img = cv.threshold(img,30,100,cv.THRESH_TOZERO)
    img = cv.bitwise_not(img)
    img = cv.blur(img,(5,5))
    _,img = cv.threshold(img,150,255,cv.THRESH_TOZERO)    
    contours, hierarchy = cv.findContours(img, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
    i = 0
    
    for cnt in contours:
        area = cv.contourArea(cnt)
        peri = cv.arcLength(cnt, True)
        approx = cv.approxPolyDP(cnt, 0.03*peri, True)         
        x, y, w, h = cv.boundingRect(approx)
        cv.putText(img,f"{len(approx)}", (x+5, y+5), cv.FONT_HERSHEY_COMPLEX,0.5,(255),2)
  
    return img

In [ ]:

display(widgets.HBox((image,origin_image,diff_image)))
i = 0 
pre_img = np.zeros((224,224),dtype=np.uint8)
while True :
    img = (camera.value) 
    img[:70,:] = 160
    hsv = frame_hue(img)
    if np.average(img) >= 100:
        img = img +20
        img[img < 0] = 0
        img[img < 60] =0
        img[img > 180] = 130
        
    img_gray= convert_to_gray(img) 
    img,app = getContours(img,img_gray)
    img_diff = frame_diff(pre_img,img_gray)
    img_gray = np.bitwise_or(img_gray,hsv[:,:,1])
    origin_image.value= bgr8_to_jpeg(img,100)
    image.value= bgr8_to_jpeg(img_gray,100)
    diff_image.value = bgr8_to_jpeg(hsv[:,:,0],100)
    time.sleep(0.1)
    pre_img = img_gray
    